In [1]:
import matplotlib.pyplot as plt
from patient_data_reader import PatientReader
import os
import time
import numpy as np
import pandas as pd

import torch

In [2]:
from config import Config

class Config:
    """feel free to play with these hyperparameters during training"""
    dataset = "resource"  # change this to the right data name
    data_path = "%s" % dataset
    checkpoint_dir = "checkpoint"
    decay_rate = 0.95
    decay_step = 1000
    n_topics = 50
    learning_rate = 0.001 # 0.00002
    vocab_size = 619
    n_stops = 22 
    lda_vocab_size = vocab_size - n_stops
    n_hidden = 200
    n_layers = 2
    projector_embed_dim = 100
    generator_embed_dim = n_hidden
    dropout = 1.0
    max_grad_norm = 1.0 #for gradient clipping
    total_epoch = 5
    init_scale = 0.075
    threshold = 0.5 #probability cut-off for predicting label to be 1
    forward_only = False #indicates whether we are in testing or training mode
    log_dir = 'logs'
    
    
FLAGS = Config()

# Define hyperparameters
N_HIDDEN = FLAGS.n_hidden
NUM_EPOCHS = FLAGS.total_epoch
N_VOCAB = FLAGS.vocab_size
EMBED_SIZE = FLAGS.projector_embed_dim
N_HIDDEN = FLAGS.n_hidden
N_TOPICS = FLAGS.n_topics
LEARNING_RATE = FLAGS.learning_rate


N_BATCH = 128 # 128, 32, 1
MAX_LENGTH = 300


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
def prepare_data(seqs, labels, vocabsize, maxlen=None):
    """Create the matrices from the datasets.

    This pad each sequence to the same lenght: the lenght of the
    longuest sequence or maxlen.

    if maxlen is set, we will cut all sequence to this maximum
    lenght.

    This swap the axis!
    """
    # x: a list of sentences
    lengths = [len(s) for s in seqs]

    eventSeq = []

    for seq in seqs:
        t = []
        for visit in seq:
            t.extend(visit)
        eventSeq.append(t)
    eventLengths = [len(s) for s in eventSeq]

    if maxlen is not None:
        new_seqs = []
        new_lengths = []
        new_labels = []
        for l, s, la in zip(lengths, seqs, labels):
            if l < maxlen:
                new_seqs.append(s)
                new_lengths.append(l)
                new_labels.append(la)
            else:
                new_seqs.append(s[:maxlen])
                new_lengths.append(maxlen)
                new_labels.append(la[:maxlen])
        lengths = new_lengths
        seqs = new_seqs
        labels = new_labels

        if len(lengths) < 1:
            return None, None, None

    n_samples = len(seqs)
    maxlen = max(maxlen, np.max(lengths)) # changed this line to always to goto max_len as we use in pytroch with batches

    x = np.zeros((n_samples, maxlen, vocabsize)).astype('int64')
    x_mask = np.zeros((n_samples, maxlen)).astype(float)
    y = np.ones((n_samples, maxlen)).astype(float)
    for idx, s in enumerate(seqs):
        x_mask[idx, :lengths[idx]] = 1
        for j, sj in enumerate(s):
            for tsj in sj:
                x[idx, j, tsj - 1] = 1
    for idx, t in enumerate(labels):
        y[idx, :lengths[idx]] = t
        # if lengths[idx] < maxlen:
        #     y[idx,lengths[idx]:] = t[-1]
    
#     # randomly generated list of labels. for testing. note that this size is n_samples,1 and not n_samples,n_visits
#     y = torch.randint(0, 2, (n_samples,)) #.astype(float)
    return x, x_mask, y, lengths, eventLengths

In [5]:
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [6]:
class CustomDataset(Dataset):
    def __init__(self, seqs, hfs):
        self.x = seqs
        self.y = hfs
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        return (self.x[index], self.y[index])
    

data_sets = PatientReader(FLAGS)

def get_custom_dataset(dtype):
    """ dtype in train, valid, test"""
    X_raw_data, Y_raw_data = data_sets.get_data_from_type(dtype)
    dataset = CustomDataset(X_raw_data, Y_raw_data)
    print(f"legth of dataset of dtype = {dtype}:", len(X_raw_data))
    return dataset

train_dataset = get_custom_dataset("train")
val_dataset = get_custom_dataset("valid")
test_dataset =get_custom_dataset("test")

 [*] load resource\vocab.pkl
 [*] load resource/X_train.pkl
 [*] load resource/Y_train.pkl
 [*] load resource/X_valid.pkl
 [*] load resource/Y_valid.pkl
 [*] load resource/X_test.pkl
 [*] load resource/Y_test.pkl
vocabulary size: 619
number of training documents: 2000
number of validation documents: 500
number of testing documents: 500
legth of dataset of dtype = train: 2000
legth of dataset of dtype = valid: 500
legth of dataset of dtype = test: 500


In [7]:
def collate_fn(data):
    """
     Collate the the list of samples into batches. For each patient, you need to pad the diagnosis
        sequences to the sample shape (max # visits, max # diagnosis codes). The padding infomation
        is stored in `mask`.
    
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.long
        masks: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.bool
        rev_x: same as x but in reversed time. This will be used in our RNN model for masking 
        rev_masks: same as mask but in reversed time. This will be used in our RNN model for masking
        y: a tensor of shape (# patiens) of type torch.float
        
    Note that you can obtains the list of diagnosis codes and the list of hf labels
        using: `sequences, labels = zip(*data)`
    """

    sequences, labels = zip(*data)

    x, x_mask, y, lengths, eventLengths = prepare_data(seqs=sequences, labels=labels, vocabsize=N_VOCAB, maxlen=MAX_LENGTH)
    
    x = torch.tensor(x, dtype=torch.float)
    x_mask = torch.tensor(x_mask, dtype=torch.bool)
    y = torch.tensor(y, dtype=torch.float)

    return x, x_mask, y, lengths, eventLengths


In [8]:
# from torch.utils.data.dataset import random_split

# split = int(len(dataset)*0.5)

# lengths = [split, len(dataset) - split]
# train_dataset, val_dataset = random_split(dataset, lengths)

# print("Length of train dataset:", len(train_dataset))
# print("Length of val dataset:", len(val_dataset))


In [9]:
from torch.utils.data import DataLoader

def load_data(train_dataset, val_dataset,test_dataset, collate_fn,batch_size=128):
    
    '''
    Implement this function to return the data loader for  train and validation dataset. 
    Set batchsize to batch_size. Set `shuffle=True` only for train dataloader.
    
    Arguments:
        train dataset: train dataset of type `CustomDataset`
        val dataset: validation dataset of type `CustomDataset`
        test dataset: test dataset of type `CustomDataset`
        
        collate_fn: collate function
        
    Outputs:
        train_loader, val_loader, test_dataset : train and validation and test dataloaders
    
    Note that you need to pass the collate function to the data loader `collate_fn()`.
    '''
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    return train_loader, val_loader, test_loader

train_loader, val_loader, test_loader = load_data(train_dataset, val_dataset,test_dataset, collate_fn,batch_size = N_BATCH)

In [10]:
def get_last_visit(hidden_states, masks):
#     print(hidden_states.shape)  # torch.Size([32, 175, 256])

#     print(masks.shape) #torch.Size([32, 175])
#     """
#      obtain the hidden state for the last true visit (not padding visits)

#     Arguments:
#         hidden_states: the hidden states of each visit of shape (batch_size, # visits, embedding_dim)
#         masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

#     Outputs:
#         last_hidden_state: the hidden state for the last true visit of shape (batch_size, embedding_dim)
        
#     NOTE: DO NOT use for loop.
    
#     HINT: First convert the mask to a vector of shape (batch_size,) containing the true visit length; 
#           and then use this length vector as index to select the last visit.
#     """

    mask_length = masks.count_nonzero(dim=1)
    return hidden_states[range(hidden_states.shape[0]),mask_length-1,:]

# GRU Net Model

In [11]:
# input = torch.randn(batch_size, sequence_length, input_size)

print_flag = False
class GRUModel(nn.Module):
    def __init__(self):
        super(GRUModel, self).__init__()
#         l_embed = lasagne.layers.DenseLayer(l_in, num_units=embedsize, b=None, num_leading_axes=2)
        self.embed = nn.Linear(N_VOCAB, EMBED_SIZE, bias=False)
        self.gru = nn.GRU(input_size=EMBED_SIZE, hidden_size=N_HIDDEN, batch_first=True)
        self.fc = nn.Linear(in_features= N_HIDDEN, out_features=MAX_LENGTH)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, masks):
        if print_flag: print( "x" ,x.shape)
        if print_flag: print( "masks",masks.shape)
        batch_size = x.shape[0]
        if print_flag: print( "batch_size", batch_size)
        x_embed = self.embed(x)
        if print_flag: print( "x_embed", x_embed.shape)
        output, h_n = self.gru(x_embed)
        if print_flag: print( "output", output.shape)
        if print_flag: print( "h_n", h_n.shape)
        true_h_n = get_last_visit(output, masks)
        if print_flag: print( "true_h_n",true_h_n.shape)
        logits = self.fc(true_h_n)   
        if print_flag: print( "logits",logits.shape)
        probs = self.sigmoid(logits)
        if print_flag: print( "probs",probs.shape)
        probs_ret = probs.view((batch_size,-1))
        if print_flag: print( "probs_ret",probs_ret.shape)
        return probs_ret
    
## H0 defaults to zeros if not provided.
#     def initHidden(self):
#         return torch.zeros(1, N_HIDDEN)

In [12]:
gru_rnn = GRUModel()
gru_rnn

GRUModel(
  (embed): Linear(in_features=619, out_features=100, bias=False)
  (gru): GRU(100, 200, batch_first=True)
  (fc): Linear(in_features=200, out_features=300, bias=True)
  (sigmoid): Sigmoid()
)

In [13]:
train_iter = iter(train_loader)
x, x_mask, y, lengths, eventLengths = next(train_iter)
x.shape, y.shape

(torch.Size([128, 300, 619]), torch.Size([128, 300]))

In [14]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(gru_rnn.parameters(), lr=0.001)

In [15]:
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score #,pr_auc


def eval_model(model, data_loader):
    
    """
    evaluate the model.
    
    Arguments:
        model: the RNN model
        val_loader: validation dataloader
        
    Outputs:
        precision: overall precision score
        recall: overall recall score
        f1: overall f1 score
        roc_auc: overall roc_auc score
        
    Note that please pass all four arguments to the model so that we can use this function for both 
    models. (Use `model(x, masks, rev_x, rev_masks)`.)
        
    HINT: checkout https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
    """
    def apply_mask(lengths, y):
        """
        for metrics need to somehow for each patient get only real n_visit not all of max_length.
            like new_testlabels.extend(inputs[1].flatten()[:leng])
        """
        result = []
        for i, l in enumerate(lengths):
            result.extend(y[i][:l])
        return result

    model.eval()
    y_pred = [] # torch.LongTensor()
    y_score = []#torch.Tensor()
    y_true = []#torch.LongTensor()
    model.eval()
    for x, x_mask, y, lengths, eventLengths in data_loader:
#         print("lengths: ", lengths)
        y_hat_prob = model(x, x_mask)
        y_score.extend(apply_mask(lengths, y_hat_prob.detach().to(device)))
        
        y_hat = (y_hat_prob > 0.5).int()
#         print(np.shape(y_pred), np.shape(y_true))
        y_pred.extend(apply_mask(lengths, y_hat.detach().to(device)))
        y_true.extend(apply_mask(lengths, y.detach().to(device)))
#         print(np.shape(y_pred), np.shape(y_true))
#         print(y_pred[:2])
        
    """
        Calculate precision, recall, f1, and roc auc scores.
        Use `average='binary'` for calculating precision, recall, and fscore.
    """
    
#     print("after loop: ", np.shape(y_pred), np.shape(y_true))
    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    roc_auc = roc_auc_score(y_true, y_score)
#     pr_auc = pr_auc(new_testlabels, pred_testlabels)
    return p, r, f, roc_auc

In [16]:
def train(model, train_loader, val_loader, n_epochs, criterion, optimizer):
    """
    train the model.
    
    Arguments:
        model: the RNN model
        train_loader: training dataloder
        val_loader: validation dataloader
        n_epochs: total number of epochs
        
    You need to call `eval_model()` at the end of each training epoch to see how well the model performs 
    on validation data.
        
    Note that please pass all four arguments to the model so that we can use this function for both 
    models. (Use `model(x, masks, rev_x, rev_masks)`.)
    """
    
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, x_mask, y, lengths, eventLengths in train_loader:
            """
                1. zero grad
                2. model forward
                3. calculate loss
                4. loss backward
                5. optimizer step
            """
            outputs = model(x, x_mask)
#             print("outputs",outputs.shape)
#             print("y",y.shape)
            loss = criterion(outputs, y) 
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        p, r, f, roc_auc = eval_model(model, val_loader)
        print('Epoch: {} \t Validation p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'
              .format(epoch+1, p, r, f, roc_auc))

In [17]:
train(gru_rnn, train_loader, val_loader, NUM_EPOCHS, criterion, optimizer)

Epoch: 1 	 Training Loss: 0.557629
Epoch: 1 	 Validation p: 0.23, r:0.32, f: 0.27, roc_auc: 0.50
Epoch: 2 	 Training Loss: 0.286657
Epoch: 2 	 Validation p: 0.23, r:0.34, f: 0.27, roc_auc: 0.50
Epoch: 3 	 Training Loss: 0.273818
Epoch: 3 	 Validation p: 0.23, r:0.32, f: 0.27, roc_auc: 0.50
Epoch: 4 	 Training Loss: 0.271648
Epoch: 4 	 Validation p: 0.23, r:0.33, f: 0.27, roc_auc: 0.50
Epoch: 5 	 Training Loss: 0.270644
Epoch: 5 	 Validation p: 0.23, r:0.33, f: 0.27, roc_auc: 0.50


In [18]:
p, r, f, roc_auc = eval_model(gru_rnn, val_loader)
print("validation roc_auc: ", roc_auc)
# assert roc_auc > 0.7, "ROC AUC is too low on the validation set (%f < 0.7)"%(roc_auc)


p, r, f, roc_auc = eval_model(gru_rnn, test_loader)
print("test roc_auc: ", roc_auc)
# assert roc_auc > 0.7, "ROC AUC is too low on the test set (%f < 0.7)"%(roc_auc)


validation roc_auc:  0.49878489869291753
test roc_auc:  0.5223959817316578


# The CONTENT Model

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# class ThetaLayer(nn.Module):
#     def __init__(self, n_topics, max_length):
#         super(ThetaLayer, self).__init__()
#         self.n_topics = n_topics
#         self.max_length = max_length
#         self.klterm = None

#     def forward(self, mu, log_sigma):
#         self.klterm = -0.5 * torch.sum(1 + 2 * log_sigma - mu.pow(2) - torch.exp(log_sigma).pow(2)) / mu.size(0)
#         eps = torch.randn(mu.size(0), self.max_length, self.n_topics).to(mu.device)
#         theta = mu.unsqueeze(1) + torch.exp(log_sigma).unsqueeze(1) * eps
#         self.theta = nn.Parameter(theta.mean(dim=0, keepdim=False))
#         return theta



class ThetaLayer(nn.Module):
    def __init__(self, incomings, maxlen, **kwargs):
        super().__init__()
        self.maxlen = maxlen
        self.logsigma = incomings[1]
        self.mu = incomings[0]
        self.theta = nn.Parameter(torch.zeros((1, self.logsigma.out_features)))
        self.register_buffer('klterm', torch.zeros((1, self.logsigma.out_features)))

    def forward(self, input):
#         logsigma_in = self.logsigma(input[0])
#         mu_in = self.mu(input[1])
        logsigma_in = self.logsigma(input)
        mu_in = self.mu(input)
        klterm = 0.5 * (1 + torch.mul(logsigma_in, 2) - (mu_in ** 2) - (torch.exp(logsigma_in) ** 2))
        self.klterm = klterm.detach()
        out = 1 / (input * (torch.exp(logsigma_in) * (2 * np.pi) ** (1 / 2))) * torch.exp(
            -((torch.log(input) - mu_in) ** 2) / (2 * (torch.exp(logsigma_in) ** 2)))
        self.theta = nn.Parameter(out.mean(dim=0, keepdim=True))
        return out
    
        # what is input?????????  
class CONTENT(nn.Module):
    def __init__(self, vocab_size, embed_size, n_hidden, n_topics, maxlen):
        super(CONTENT, self).__init__()
        
        # embed layer to reduce dimensionality
        self.l_embed = nn.Linear(vocab_size, embed_size, bias=False)
        
        # GRU
        self.l_forward0 = nn.GRU(embed_size, n_hidden, batch_first=True, bidirectional=False)

        # Recognition Net
        self.l_1 = nn.Linear(vocab_size, n_hidden)
        self.l_2 = nn.Linear(n_hidden, n_hidden)
        
        self.mu = nn.Linear(n_hidden, n_topics)
        self.log_sigma = nn.Linear(n_hidden, n_topics)
#         self.l_theta = ThetaLayer(n_topics, maxlen)
        self.l_theta = ThetaLayer([self.mu, self.log_sigma], maxlen)

        
        self.l_B = nn.Linear(vocab_size, n_topics, bias=False) 
        self.l_context = nn.Sequential(
            nn.BatchNorm1d(n_topics),
            nn.Linear(n_topics, n_topics)
        )
        self.l_dense0 = nn.Linear(n_hidden, 1)
        self.l_dense1 = nn.Flatten(start_dim=1)
        self.l_dense = nn.Sequential(
            nn.Linear(n_topics + maxlen, 1),
            nn.Flatten(start_dim=1),
        )
        

    def forward(self, x, mask):
        print("x", x.shape)
        
        x_emb = self.l_embed(x)
        print("x_emb",x_emb.shape)
        
        x_forward, _ = self.l_forward0(x_emb)
        print("mask",mask.shape)
        
        x_forward = x_forward * mask.unsqueeze(2)
        print("x_forward",x_forward.shape)

        x_1 = F.relu(self.l_1(x))
        x_2 = F.relu(self.l_2(x_1))
        mu = self.mu(x_2)
        log_sigma = self.log_sigma(x_2)
        print("mu",mu.shape)
        print("log_sigma",log_sigma.shape)
        
#         theta = self.l_theta(mu, log_sigma)
        theta = self.l_theta(x_2)

        print("theta",theta.shape)
        

        b = self.l_B(x)
        print("b",b.shape)
        
        context = b * theta
        print("context before mean",context.shape)
        context = torch.mean(context, dim=-1)
#         context = b * theta
        print("context",context.shape)
        
        # combine GRU and context

        x_dense0 = self.l_dense0(x_forward)
        print("x_dense0",x_dense0.shape)
        
        x_dense1 = self.l_dense1(x_dense0)
        print("x_dense1",x_dense1.shape)
        x_dense = self.l_dense(torch.cat((x_dense1, context), dim=1))
        print("x_dense",x_dense.shape)

        output = torch.sigmoid(x_dense) * mask + 0.000001
        print("output",output.shape)
        out = output.view(-1, maxlen)
        print("out",out.shape)
        
        return out
    
    
# Define model and optimizer
content_model = CONTENT(N_VOCAB, EMBED_SIZE, N_HIDDEN, N_TOPICS, MAX_LENGTH)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(content_model.parameters(),lr=LEARNING_RATE)
train(content_model, train_loader, val_loader, NUM_EPOCHS, criterion, optimizer)

x torch.Size([128, 300, 619])
x_emb torch.Size([128, 300, 100])
mask torch.Size([128, 300])
x_forward torch.Size([128, 300, 200])
mu torch.Size([128, 300, 50])
log_sigma torch.Size([128, 300, 50])


RuntimeError: The size of tensor a (200) must match the size of tensor b (50) at non-singleton dimension 2

In [ ]:
train_iter = iter(train_loader)
x, x_mask, y, lengths, eventLengths = next(train_iter)
x.shape, y.shape

#3 

In [ ]:
# Todo: need to write cost function which include klterm as well

In [ ]:
# from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, accuracy_score
# import time
# from tqdm import tqdm

# # define the training function
# def train(model, optimizer, criterion, x, y, mask):
#     optimizer.zero_grad()
#     output = model(x, mask)
#     loss = criterion(output.flatten(), y)
#     loss.backward()
#     optimizer.step()
#     return loss.item()

# # define the function to compute cost
# def compute_cost(model, x, y, mask):
#     output = model(x, mask)
#     loss = nn.BCELoss()(output.flatten(), y)
#     return loss.item()

# # define the function to output the hidden state
# def output_theta(model, x, mask):
#     with torch.no_grad():
#         model.eval()
#         output, hn = model.rnn(x)
#         output = output * mask.unsqueeze(-1)
#         output = output.sum(dim=1) / mask.sum(dim=1).unsqueeze(-1)
#     return hn.detach().numpy().reshape(x.shape[0], -1), output.detach().numpy()


